# Assignment 7

### Part A

In [90]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [91]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [92]:
def expected_return(P, rf, mkt_rp):

    total_invested = P['Amount Invested'].sum()
    P['Weight'] = P['Amount Invested']/total_invested
    P["Temp"] = P['Weight']*P['Beta']
    beta_p = P['Temp'].sum()
    E_Rp = rf + beta_p*mkt_rp

    return E_Rp

expected_return(portfolio, 0.02, 0.05)

0.086

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [93]:
def volatility(P, cor):
    P['Weight'] = P['Amount Invested'] / P['Amount Invested'].sum()

    portfolio_variance = 0
    for i in range(len(P)):
        for j in range(i, len(P)):
            if i == j:  # Variance term when i and j are the same
                portfolio_variance += (P['Weight'][i] ** 2) * (P['Volatility'][i] ** 2)
            else:  # Covariance term when i and j are different
                portfolio_variance += 2 * P['Weight'][i] * P['Weight'][j] * P['Volatility'][i] * P['Volatility'][j] * cor

    return portfolio_variance ** 0.5

volatility(portfolio, -0.2)

0.24083189157584592

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [94]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    w = vol / mkt_vol # marketportfolio weight

    expected_return_market = rf + mkt_rp
    expected_return_portfolio = w * expected_return_market + (1 - w) * rf

    return expected_return_portfolio

expected_return2(0.02, 0.05, 0.16, 0.2408)

0.09525

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [95]:
def CAPM_beta(return_path, factors_path):
    # Read the data
    returns = pd.read_csv(return_path)
    factors = pd.read_csv(factors_path)

    # Convert factors percentages to decimals
    factors['Mkt-RF'] /= 100
    factors['RF'] /= 100

    # Merge the data on year and month
    returns['date'] = returns['year'].astype(str) + returns['month'].astype(str).str.zfill(2)
    factors['date'] = factors['date'].astype(str)
    merged_data = pd.merge(returns, factors, on='date', how='inner')
    merged_data = merged_data.drop(['year', 'month', 'date'], axis=1)

    # Calculate excess returns
    merged_data['Excess_RET'] = merged_data['RET'] - merged_data['RF']

    # Group by CUSIP and calculate Beta
    output = pd.DataFrame(columns=["CUSIP", "EXP_RETURN", "BETA"])

    for name, group in merged_data.groupby('CUSIP'):
        cov = (group['Excess_RET']).cov(group['Mkt-RF'])
        var = group['Mkt-RF'].var()
        beta = cov / var 
        exp_return = group['RET'].mean()
        temp = pd.DataFrame({'CUSIP': [name], 'EXP_RETURN': [exp_return], 'BETA': [beta]})
        output = pd.concat([output, temp], ignore_index=True)

    output.reset_index(drop=True, inplace=True)
    return output

CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.CSV")

C:\Users\wzlwx\AppData\Local\Temp\ipykernel_31824\3861020241.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output = pd.concat([output, temp], ignore_index=True)


CUSIP  EXP_RETURN      BETA
0     00081T10    0.016252  2.591073
1     00108M10    0.011101  0.741057
2     00130H10    0.016152  1.457100
3     00163U10    0.015538  0.975530
4     00182C10    0.014772  1.406085
...        ...         ...       ...
3220   9959910    0.009933  0.763107
3221   9972410    0.012866  1.280548
3222   9972510    0.017177  1.065298
3223   9984910    0.009006  1.876418
3224   9985510    0.015699  1.166459

[3225 rows x 3 columns]

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

This should return 0.007981588120535642... with the provided data.

In [96]:
def slope(data):
    beta = data['BETA']
    exp_return = data['EXP_RETURN']
    slope = np.polyfit(beta, exp_return, 1)[1]

    return slope

slope(CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.CSV"))

C:\Users\wzlwx\AppData\Local\Temp\ipykernel_31824\3861020241.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output = pd.concat([output, temp], ignore_index=True)


0.007981588120535638